In [38]:
import numpy as np
import keras
from sklearn.model_selection import KFold

In [2]:
training_data = np.loadtxt('../data/training_data.txt', skiprows=1)
X_test = np.loadtxt('../data/test_data.txt', skiprows=1)

In [39]:
X_train = training_data[:,1:]
y_train = training_data[:,0]
y_train_cat = keras.utils.to_categorical(y_train)
kf = KFold(n_splits=4, shuffle=True, random_state=214)

In [32]:
# Adaboost
tr1 = np.loadtxt('../inferences/adaboost_train.txt', ndmin=2)

# KNN
tr2 = np.loadtxt('../inferences/knn_2_train.txt', ndmin=2)
tr3 = np.loadtxt('../inferences/knn_4_train.txt', ndmin=2)
tr4 = np.loadtxt('../inferences/knn_8_train.txt', ndmin=2)
tr5 = np.loadtxt('../inferences/knn_16_train.txt', ndmin=2)
tr6 = np.loadtxt('../inferences/knn_32_train.txt', ndmin=2)
tr7 = np.loadtxt('../inferences/knn_64_train.txt', ndmin=2)
tr8 = np.loadtxt('../inferences/knn_128_train.txt', ndmin=2)
tr9 = np.loadtxt('../inferences/knn_256_train.txt', ndmin=2)
tr10 = np.loadtxt('../inferences/knn_512_train.txt', ndmin=2)
tr11 = np.loadtxt('../inferences/knn_1024_train.txt', ndmin=2)

# Random Forest
tr12 = np.loadtxt('../inferences/rf_train.txt', ndmin=2)
tr13 = np.loadtxt('../inferences/rf2_train.txt', ndmin=2)
tr14 = np.loadtxt('../inferences/rf3_train.txt', ndmin=2)

# LibFFM
tr15 = np.loadtxt('../inferences/train_ffm.txt', ndmin=2)

# Vowpal Wabbit
tr16 = np.loadtxt('../inferences/train_vw_logistic.txt', ndmin=2)
te17 = np.loadtxt('../inferences/train_vw_logistic_nn5.txt')

# XGBoost
tr18 = np.loadtxt('../inferences/xgb_train.txt', ndmin=2)
tr19 = np.loadtxt('../inferences/xgb2_train.txt', ndmin=2)


# Adaboost
te1 = np.loadtxt('../inferences/adaboost_test.txt', ndmin=2)

# KNN
te2 = np.loadtxt('../inferences/knn_2_test.txt', ndmin=2)
te3 = np.loadtxt('../inferences/knn_4_test.txt', ndmin=2)
te4 = np.loadtxt('../inferences/knn_8_test.txt', ndmin=2)
te5 = np.loadtxt('../inferences/knn_16_test.txt', ndmin=2)
te6 = np.loadtxt('../inferences/knn_32_test.txt', ndmin=2)
te7 = np.loadtxt('../inferences/knn_64_test.txt', ndmin=2)
te8 = np.loadtxt('../inferences/knn_128_test.txt', ndmin=2)
te9 = np.loadtxt('../inferences/knn_256_test.txt', ndmin=2)
te10 = np.loadtxt('../inferences/knn_512_test.txt', ndmin=2)
te11 = np.loadtxt('../inferences/knn_1024_test.txt', ndmin=2)

# Random Forest
te12 = np.loadtxt('../inferences/rf_test.txt', ndmin=2)
te13 = np.loadtxt('../inferences/rf2_test.txt', ndmin=2)
te14 = np.loadtxt('../inferences/rf3_test.txt', ndmin=2)


# LibFFM
te15 = np.loadtxt('../inferences/test_ffm.txt', ndmin=2)


# Vowpal Wabbit
te16 = np.loadtxt('../inferences/test_vw_logistic.txt', ndmin=2)
te17 = np.loadtxt('../inferences/test_vw_logistic_nn5.txt', ndmin=2)

# XGBoost
te18 = np.loadtxt('../inferences/xgb_test.txt', ndmin=2)
te19 = np.loadtxt('../inferences/xgb2_test.txt', ndmin=2)

In [62]:
X_meta_train = np.hstack([tr1, tr2, tr3, tr4, tr5, tr6, tr7, tr8, tr9, tr10, tr11, tr12, tr13, tr14, tr15, tr16, tr17, tr18])
X_meta_test = np.hstack([te1, te2, te3, te4, te5, te6, te7, te8, te9, te10, te11, te12, te13, te14, te15, te16, te17, te18])

In [35]:
X_meta_test.shape

(10000, 38)

In [44]:
from keras.layers import *
from keras.layers.core import *
from keras.models import Sequential, Model

In [60]:
def build_model():
    inp1 = keras.layers.Input(shape=(38, ))
    x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(inp1)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = Dropout(0.5)(x)
    
    inp2 = keras.layers.Input(shape=(1000, ))
    
    x = concatenate([x, inp2])    


    #   a = GlobalMaxPooling1D()(v)
    #   b = GlobalAveragePooling1D()(v)
    #   c = GlobalMinPooling1D()(v)
    #   x = keras.layers.concatenate([a,b,c])
#     x = Flatten()(x)
#     x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
#     x = Dropout(0.8)(x)
    x = Dense(2, activation='softmax')(x)

    model = Model(inputs=[inp1,inp2], outputs=x)
    #   model.summary()
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model

In [63]:
model = build_model()
model.summary() 
h = model.fit([X_meta_train, X_train], y_train_cat, epochs=20, batch_size=32, validation_split=0.25)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 38)           0                                            
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 64)           2496        input_5[0][0]                    
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, 64)           0           dense_15[0][0]                   
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 64)           4160        dropout_11[0][0]                 
__________________________________________________________________________________________________
dropout_12

In [55]:
y_test = model.predict(X_meta_test)

In [56]:
y_test = np.array([0 if r0 > r1 else 1 for r0, r1 in y_test])

In [57]:
np.savetxt('test_output.txt', np.dstack((np.arange(1, y_test.shape[0] + 1), y_test.T))[0], "%d,%d", delimiter=',', header="Id,Prediction", comments="")